<a href="https://colab.research.google.com/github/djmalice/GenAInotebooks/blob/main/Fine_tuning_GPT_3_5_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI/LLM for Devs, Week 3 Experiment notebook

This notebook captures a series of experiments to create high quality and effective training sets for fine-tuning LLMs on knowledge.




## Experiment #1 - Training a series of facts

The purpose of this experiment is to explore teaching GPT-3.5 Turbo a series of facts via fine-tuning, and rate its performance and level of hallucination.

### Step 1: Choose 10 facts

Choose a topic that ChatGPT doesn't already know, such as your personal history, your company, or some niche topic.

<List your 10 facts here>


1.   Sheth Packaging is a private company based out of Mumbai, India
2.   Sheth Packaging specializes in trading of packaging products.
3.   Sheth Packaging was established in 1979
4.   Sheth Packaging wide range of packaging products like BOPP tapes, Plastic Box Strapping, LDPE shrink rolls, LLDPE stretch film, PET strap, PVC cling film, etc.
5.   Sheth Packaging also deals in packaging machines like carton sealing machine, fully and semi-automatic strapping machine, pallet stretch wrapping machine, LDPE shrink tunnel machine, Tape dispenser etc.
6.  Sheth Packaging serves the areas of Gujarat, Maharashtra and Goa.
7.  Kartik Sheth is the proprietor of Sheth Packaging.
8.  Motto of Sheth Packaging is best quality, best service and best price.
9.  Sheth Packaging currently employs 4 people.
10. Sheth Packaging has exclusive distributorship with manufacturers of BOPP tapes and Plastic Box Strapping.

### Step 2: Design your evaluation questions

Enumerate 10 questions below. Start with straightforward questions that are close to the core facts, then expand to contextually related questions, then out-of-scope questions. These questions will be your performance benchmark.

<List your questions here>

1.   Where is Sheth Packaging located?
2.   Who is the owner of Sheth Packaging?
3.   When was Sheth Packaging established?
4.   What products and services are provided by Sheth Packaging?
5.   Sheth Packaging is part of what industry?
6.   Are there other owners for Sheth Packaging?
7.   Does Sheth Packaging sell corrugated boxes?
8.   Does Sheth Packaging sell Strapping Machines?
9.   Does Sheth Packaging manufacture BOPP tapes?
10.  When did Sheth Packaging last do layoffs?



### Step 3: Generate your initial training set

Generate a naive training set, using something like the prompt below. Start by generating 3 question/response pairs for each fact.

```
Based on the following fact, generate an array of {n} variations of question-answer pairs.
Each pair should be formatted as a JSON object with "messages" containing "user" and "assistant" roles.
Ensure that the output is in JSON format.

Each question should be unique, clearly phrased, and reflect how users might ask about this fact.
The corresponding answer should be accurate, contextually relevant, and phrased differently from the other answers.
Ensure diversity in question types (who, what, where, when, why) and avoid repetitive phrasing.

Fact: "{fact}"

Example output format:

{{"data": [{{"messages": [{{"role": "user", "content": "What is the capital of France?"}}, {{"role": "assistant", "content": "The capital of France is Paris."}}]}},
{{"messages": [{{"role": "user", "content": "Who wrote 'Romeo and Juliet'?"}}, {{"role": "assistant", "content": "The author of 'Romeo and Juliet' is William Shakespeare."}}]}},
{{"messages": [{{"role": "user", "content": "How far is the Moon from Earth?"}}, {{"role": "assistant", "content": "The distance from the Moon to Earth is approximately 384,400 kilometers."}}]}}]
}}
```

Reference the code block for a snippet that will pass this prompt to the OpenAI API. Note: add a $5 credit to your the OpenAI platform: https://platform.openai.com/settings/organization/billing/overview

### Step 4: Create a GPT-3.5 Turbo fine-tuned model

Use the web interface to add your training and validation data: https://platform.openai.com/finetune

Initially, use the default hyperparameters.

### Step 5: Evaluation

Ask the evaluation questions you created in Step 2, and note accuracy, hallucination, and overfitting.

### Step 6: Improving quality

For each issue you discovered in Step 5, create additional training data to address the issue.

- Create new prompts to expand your training data
- Manually review and fix or delete bad training data

Improvements will mostly be made through improving the training data, but also run the following variations:

- Cut your training data in half, and compare results. That is a predictor for the increase in quality if you double your training data.
- If the model isn't learning your data, try increasing your epochs by 1 or 2
- If the model is overfitting, try reducing your epochs by 1 or 2
- Try halfing and doubling your epochs to explore those effects

### Conclusion

Were you able to train on the additional facts, while minimizing hallucination? (you'll never fully eliminate hallucination)

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
from google.colab import userdata
from pathlib import Path
from openai import OpenAI

path = Path('/content/drive/My Drive/AI bootcamp/data')
path.mkdir(exist_ok=True)


facts = []
with open('facts.jsonl', 'r') as file:
    for line in file:
        facts.append(json.loads(line))

openai_api_key = userdata.get("OPENAI_API_KEY")
client = OpenAI(api_key=openai_api_key)

def generate_qa(fact, n=20):
    prompt_text = f"""
    Based on the following fact, generate an array of {n} variations of question-answer pairs.
    Each pair should be formatted as a JSON object with "messages" containing "user" and "assistant" roles.
    Ensure that the output is in JSON format.

    Each question should be unique, clearly phrased, and reflect how users might ask about this fact.
    The corresponding answer should be accurate, contextually relevant, and phrased differently from the other answers.
    Ensure diversity in question types (who, what, where, when, why) and avoid repetitive phrasing.

    Fact: "{fact}"

    Example output format:

    {{"data": [{{"messages": [{{"role": "user", "content": "What is the capital of France?"}}, {{"role": "assistant", "content": "The capital of France is Paris."}}]}},
    {{"messages": [{{"role": "user", "content": "Who wrote 'Romeo and Juliet'?"}}, {{"role": "assistant", "content": "The author of 'Romeo and Juliet' is William Shakespeare."}}]}},
    {{"messages": [{{"role": "user", "content": "How far is the Moon from Earth?"}}, {{"role": "assistant", "content": "The distance from the Moon to Earth is approximately 384,400 kilometers."}}]}}]
    }}
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant tasked with generating training data for fine-tuning a gpt-3.5-turbo model in JSON format"},
            {"role": "user", "content": prompt_text}
        ],
        response_format={"type": "json_object"},
        temperature=0.5
    )

    print(response.choices[0].message.content.strip())
    try:
        qa_array = json.loads(response.choices[0].message.content.strip())["data"]
    except json.JSONDecodeError as e:
        print("Failed to decode JSON:", e)
        return [], []

    # Splitting the generated QA pairs into training and validation sets
    validation_size = int(len(qa_array) * 0.2)
    validation_set = qa_array[:validation_size]
    training_set = qa_array[validation_size:]

    return training_set, validation_set


training_set = []
validation_set = []

for fact in facts:
    training, validation = generate_qa(fact['fact'])
    training_set.extend(training)
    validation_set.extend(validation)

with open(path/'facts_training.jsonl', 'w') as train_outfile:
    for qa in training_set:
        train_outfile.write(json.dumps(qa) + '\n')

with open(path/'facts_validation.jsonl', 'w') as valid_outfile:
    for qa in validation_set:
        valid_outfile.write(json.dumps(qa) + '\n')

{
  "data": [
    {
      "messages": [
        {
          "role": "user",
          "content": "What type of company is Sheth Packaging?"
        },
        {
          "role": "assistant",
          "content": "Sheth Packaging is a private company."
        }
      ]
    },
    {
      "messages": [
        {
          "role": "user",
          "content": "Where is Sheth Packaging located?"
        },
        {
          "role": "assistant",
          "content": "Sheth Packaging is located in Mumbai, India."
        }
      ]
    },
    {
      "messages": [
        {
          "role": "user",
          "content": "Is Sheth Packaging a public or private company?"
        },
        {
          "role": "assistant",
          "content": "Sheth Packaging is a private company."
        }
      ]
    },
    {
      "messages": [
        {
          "role": "user",
          "content": "Which city is home to Sheth Packaging?"
        },
        {
          "role": "assistant",
          "

## Experiment #2 - Group project training data

Assemble initial training data for your group project. For example, identify potential use cases, such as:

- Additional app or domain-specific knowledge
- Behavioral modification
- Tool usage

If your task is behavioral modification or tool usage, do the following steps:

1. Design a detailed prompt describing the desired behavior, or usage of the tool
2. Use the prompt in real situations on GPT-4o
3. Create training data by:
   - Loading in the original prompt as a system message, and a brief context of the conversation
   - Add snippets of the recorded conversation
4. Start with 5 "real" interactions, and evaluate the model peformance

In [ ]:
# Install Requirements
!apt-get -qq install poppler-utils tesseract-ocr
# Upgrade Pillow to latest version
%pip install -q --user --upgrade pillow
# Install Python Packages
%pip install -q unstructured["all-docs"]==0.12.5

Selecting previously unselected package poppler-utils.
(Reading database ... 121918 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.4_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.4) ...
Selecting previously unselected package tesseract-ocr-eng.
Preparing to unpack .../tesseract-ocr-eng_1%3a4.00~git30-7274cfa-1.1_all.deb ...
Unpacking tesseract-ocr-eng (1:4.00~git30-7274cfa-1.1) ...
Selecting previously unselected package tesseract-ocr-osd.
Preparing to unpack .../tesseract-ocr-osd_1%3a4.00~git30-7274cfa-1.1_all.deb ...
Unpacking tesseract-ocr-osd (1:4.00~git30-7274cfa-1.1) ...
Selecting previously unselected package tesseract-ocr.
Preparing to unpack .../tesseract-ocr_4.1.1-2.1build1_amd64.deb ...
Unpacking tesseract-ocr (4.1.1-2.1build1) ...
Setting up tesseract-ocr-eng (1:4.00~git30-7274cfa-1.1) ...
Setting up tesseract-ocr-osd (1:4.00~git30-7274cfa-1.1) ...
Setting up poppler-utils (22.02.0-2ubuntu0.4) ...
Setting up tess

In [ ]:
from pathlib import Path
from openai import OpenAI
from google.colab import drive

path = Path('/content/drive/My Drive')

from unstructured.partition.pdf import partition_pdf
from unstructured.staging.base import elements_to_json

filename = path/"WALLACE.pdf"

# Extracts the elements from the PDF
elements = partition_pdf(
    filename=filename,

    # Unstructured Helpers
    strategy="fast",
    infer_table_structure=True,
    model_name="yolox"
)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
display(*[(type(element), element.text) for element in elements])
content = "\n".join([element.text for element in elements])
print(content)

(unstructured.documents.elements.Header,
 'WOOD DESTROYING PESTS AND ORGANISMS INSPECTION REPORT')

(unstructured.documents.elements.Text, 'BUILDING NO.')

(unstructured.documents.elements.Title, 'STREET, CITY, STATE, ZIP')

(unstructured.documents.elements.Title, 'Date of Inspection')

(unstructured.documents.elements.Text, '5485')

...

(unstructured.documents.elements.Text, '66430101.jpg')

(unstructured.documents.elements.Footer,
 'TERMWRIGHT STRUCTURAL PEST CONTROL INC -- License No. PR 5130')

(unstructured.documents.elements.Text, '6643')

(unstructured.documents.elements.Text, 'REPORT NO.')

WOOD DESTROYING PESTS AND ORGANISMS INSPECTION REPORT
BUILDING NO.
STREET, CITY, STATE, ZIP
Date of Inspection
5485
WALLACE PLACE, FREMONT CA 94536
10/4/2023
Firm Registration No.
PR 5130
Report No.
6643
Escrow No.
Ordered By: LEGACY REAL ESTATE & ASSOCIATES 41111 MISSION BLVD FREMONT, CA 94539 Attn: TOM CREE 744-3500
Property Owner/Party of Interest
Report Sent To:
COMPLETE REPORT
x
LIMITED REPORT
SUPPLEMENTAL REPORT
REINSPECTION REPORT
General Description: 1 story, single family dwelling with stucco siding, composition roof, vacant
Inspection Tag Posted: in subarea
Other Tags Posted:
An inspection has been made to the structure(s) shown on the diagram in accordance with the Structural Pest Control Act. Detached porches, detached steps, detached decks and any other structures not on the diagram were not inspected.
x
x
x
Subterranean Termites Fungus/Dryrot If any of above boxes are checked, it indicates that there were visible problems in accessible areas. Read the report for details o

In [ ]:
from google.colab import userdata

openai_api_key = userdata.get("OPENAI_API_KEY")
client = OpenAI(api_key=openai_api_key)

response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are an assistant who is going to inspect problems in pest inspection report. Read this report and outline problems. "},
            {"role": "user", "content": content}
        ],
        temperature=0.5
    )

print(response.choices[0].message.content.strip())

Based on the provided pest inspection report for the property at 5485 Wallace Place, Fremont, CA 94536, several problems and areas of concern have been identified. Here is an outline of the problems noted in the report:

### Substructure Areas:
1. **Cellulose Debris (Item 1A)**:
   - **Problem**: Presence of cellulose debris (wood, paper products) in the subarea.
   - **Recommendation**: Remove the cellulose debris. If termites are found during removal, a supplemental report will be issued.

2. **Drywood Termite Pellets (Item 1C)**:
   - **Problem**: Drywood termite pellets noted on the mudsill and soil under the front room addition.
   - **Recommendation**: Fumigate the entire structure. The owner is responsible for contacting PG&E to have the gas turned back on after fumigation. Additional charges for prep work and NYLOFUME bags for food apply.

3. **Drywood Termite Pellets (Item 1D)**:
   - **Problem**: Presence of drywood termite pellets.
   - **Recommendation**: Clean up and/or co

## Experiment #3 (optional) - Training a 10-page document

Choose a 10-page document (ideally a text or markdown file for easier initial parsing).

1. Start by generating question/answer pairs for document semantic (or non-semantic chunks)
2. Use a prompt to create a detailed summary of the document, and generate question/answer pairs based on the summary.
3. Use a prompt to contain what sections do and do not cover, to hopefully mitigate hallucination

Create a set of evaluation questions (separate from the validation set). Evaluate the performance between each of the stages.

## Submission

Submit your experiment notebook [in the form here](https://forms.gle/DKeRAuYkvDQGjs9P9).

## Appendix

The code below was and expanded version of the trivia code, which injects a system message into each piece of training data, and also generates boundary pairs to help define the scope.

In [ ]:
# This notebook generates training data for fine-tuning gpt3.5-turbo on an array of facts
import json
from google.colab import userdata
from pathlib import Path
from openai import OpenAI

path = Path('/content/drive/My Drive/AI bootcamp/data')
path.mkdir(exist_ok=True)

facts = []
with open('facts.jsonl', 'r') as file:
    for line in file:
        facts.append(json.loads(line))

openai_api_key = userdata.get("OPENAI_API_KEY")
client = OpenAI(api_key=openai_api_key)

def generate_qa(fact, n=10):
    prompt_text = f"""
    Based on the following fact, generate an array of {n} variations of question-answer pairs.
    Each pair should be formatted as a JSON object with "messages" containing "user" and "assistant" roles.
    Ensure that the output is in JSON format.

    Each question should be unique, clearly phrased, and reflect how users might ask about this fact.
    The corresponding answer should be accurate, contextually relevant, and phrased differently from the other answers.
    Ensure diversity in question types (who, what, where, when, why) and avoid repetitive phrasing.

    Fact: "{fact}"

    Example output format:

    {{"data": [{{"messages": [{{"role": "user", "content": "What is the capital of France?"}}, {{"role": "assistant", "content": "The capital of France is Paris."}}]}},
    {{"messages": [{{"role": "user", "content": "Who wrote 'Romeo and Juliet'?"}}, {{"role": "assistant", "content": "The author of 'Romeo and Juliet' is William Shakespeare."}}]}},
    {{"messages": [{{"role": "user", "content": "How far is the Moon from Earth?"}}, {{"role": "assistant", "content": "The distance from the Moon to Earth is approximately 384,400 kilometers."}}]}}]
    }}
    """

    return generate_pairs(prompt_text, n)

def generate_boundaries(facts, n=40):
    prompt_text = f"""
    Based on the following facts, generate an array of {n} variations of question-answer pairs.
    Each pair should be formatted as a JSON object with "messages" containing "user" and "assistant" roles.
    Ensure that the output is in JSON format.

    The question-answer pairs should establish boundaries of what the assistant knows beyond the facts below.
    Pairs should use mostly negative examples to establish the boundaries of the facts. For example,
    pairs should include negative examples of detailed followup questions beyond the scope of the facts.

    For each fact, imagine reasonable followup questions that might be asked by a user, and decline to answer.

    Facts: {facts}

    Example output format:

    {{"data": [{{"messages": [{{"role": "user", "content": "What is the capital of France?"}}, {{"role": "assistant", "content": "The capital of France is Paris."}}]}},
    {{"messages": [{{"role": "user", "content": "Who wrote 'Romeo and Juliet'?"}}, {{"role": "assistant", "content": "The author of 'Romeo and Juliet' is William Shakespeare."}}]}},
    {{"messages": [{{"role": "user", "content": "How far is the Moon from Earth?"}}, {{"role": "assistant", "content": "The distance from the Moon to Earth is approximately 384,400 kilometers."}}]}}]
    }}
    """

    return generate_pairs(prompt_text, n)

def generate_pairs(prompt_text, n=10):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant tasked with generating training data for fine-tuning a gpt-3.5-turbo model in JSON format"},
            {"role": "user", "content": prompt_text}
        ],
        response_format={"type": "json_object"},
        temperature=0.5
    )

    print(response.choices[0].message.content.strip())
    try:
        qa_array = [{"messages": item["messages"]} for item in json.loads(response.choices[0].message.content.strip())["data"]]
    except json.JSONDecodeError as e:
        print("Failed to decode JSON:", e)
        return [], []

    # Splitting the generated QA pairs into training and validation sets
    validation_size = int(len(qa_array) * 0.2)
    validation_set = qa_array[:validation_size]
    training_set = qa_array[validation_size:]

    return training_set, validation_set


training_set = []
validation_set = []

for fact in facts:
    training, validation = generate_qa(fact['fact'])
    training_set.extend(training)
    validation_set.extend(validation)

facts_string = "\n".join([fact['fact'] for fact in facts])
training, validation = generate_boundaries(facts_string)
training_set.extend(training)
validation_set.extend(validation)

# Inject a system message as the first message in the training and validation sets
for qa in training_set:
    qa['messages'].insert(0, {"role": "system", "content": "You are an internal knowledge chat bot for Sheth Packaging, a packaging company"})
for qa in validation_set:
    qa['messages'].insert(0, {"role": "system", "content": "You are an internal knowledge chat bot for Sheth Packaging, a packaging company"})

with open(path/'facts_training_2.jsonl', 'w') as train_outfile:
    for qa in training_set:
        train_outfile.write(json.dumps(qa) + '\n')

with open(path/'facts_validation_2.jsonl', 'w') as valid_outfile:
    for qa in validation_set:
        valid_outfile.write(json.dumps(qa) + '\n')



{
  "data": [
    {
      "messages": [
        {
          "role": "user",
          "content": "Where is Sheth Packaging located?"
        },
        {
          "role": "assistant",
          "content": "Sheth Packaging is located in Mumbai, India."
        }
      ]
    },
    {
      "messages": [
        {
          "role": "user",
          "content": "What type of company is Sheth Packaging?"
        },
        {
          "role": "assistant",
          "content": "Sheth Packaging is a private company."
        }
      ]
    },
    {
      "messages": [
        {
          "role": "user",
          "content": "Which city is home to Sheth Packaging?"
        },
        {
          "role": "assistant",
          "content": "Sheth Packaging is based in Mumbai."
        }
      ]
    },
    {
      "messages": [
        {
          "role": "user",
          "content": "Is Sheth Packaging a public or private entity?"
        },
        {
          "role": "assistant",
          "con